In [ ]:
# HyperParameter tunning using Optuna and CNN implementation

import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn

In [ ]:
class CustomDatest(Dataset):
  def __init__(self,features,labels):

    self.features=torch.tensor(features,dtype=torch.float32,device=device)
    self.labels=torch.tensor(features,dtype=torch.long,device=device)

  def __len__(self,features):
    return len(self.features)


  def __getitem__(self, index):
    return self.features[index],self.labels[index]

In [ ]:
training_data=CustomDataset(X_train,y_train)
test_data=CustomDataset(X_test,y_test)

In [ ]:
training_data_load=DataLoader(dataset=dataset,shuffle=True,batch_size=32,pin_memory=True)
test_data_load=DataLoader(dataset=test_data,shuffle=False,batch_size=32,pin_memory=True)

In [ ]:
device=torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [ ]:


class FlexibileMyNN(nn.Moemory=True)dule):

  def __init__(self,input_dims,output_dims,hidden_number_layers,neuron_per_layer,dropout_rate):

    #calling the parent class constructor
    super().__init__()

    #making  the model

    #collecting all the layers in a list
    layers=[]

    #looping throught the hidden layers
    #in each iteration we add the hidden layer to the layers list
    for i in range(hidden_number_layers):

      layers.append(nn.Linear(input_dims,neuron_per_layer))
      layers.append(nn.BatchNorm1d(neuron_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      #making sure tat in the next loop the outputlayer becomes the input
      input_dims=neuron_per_layer

    layers.append(nn.Linear(neuron_per_layer,output_dims))

    self.model=nn.Sequential(*layers)

    #forward props
  def forward(self,features):

      output=self.model(features)

      return output


In [ ]:

def objective_function(trial):

  #next hyperparams suggestions
  neuron_per_layer=trial.suggest_int('neurons',8,264,step=8)
  hidden_number_layers=trial.suggest_int('hidden_layers',1,10)
  epochs=trial.suggest_int('epochs',20,120,step=20)
  learning_rate=trial.suggest_int('lr',0.1,0.5)
  weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2,log=True)
  dropout_rate=trial.suggest_int('droput',2,6,step=1)

  #input and output_dims

  input_dims=784
  output_dims=10

  #model init

  model=FlexibileMyNN(input_dims,output_dims=output_dims,hidden_number_layers=hidden_number_layers,neuron_per_layer=neuron_per_layer,dropout_rate=dropout_rate)

  model.to(device)

  #params

  learning_rate=learning_rate

  epochs=epochs

  loss=nn.CrossEntropyLoss()

  #optimizer selection

  optimizer=torch.optim.SGD(params=model.parameters(),weight_decay=weight_decay,lr=learning_rate)

  #dataset_loading

  training_data_load=DataLoader(dataset=dataset,shuffle=True,batch_size=32,pin_memory=True)
  test_data_load=DataLoader(dataset=test,shuffle=False,batch_size=32,pin_memory=True)

  #training loop

  for i in range(epochs):

    for batch_features,batch_labels in training_data_load:

      batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)

      #forward_propagation
      prediction=model(batch_features)

      #lloss_calculation
      loss_output=loss(prediction,batch_labels)

      #zero_optimizers
      optimizer.zero_grad()

      #backward_prop
      loss_output.backward()

      #Optimize
      optimizer.step()

  #evaluation

  model.eval()

  total=0
  correct=0

  #evaluation loop on test data


  with torch.no_grad():

    for  batch_features,batch_labels in test_data_load:

      batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)

      #forward_propagation
      prediction=model(batch_features)

      #finding the maximum probability
      _,prediction=torch.max(prediction,1)

      #total size
      total+=batch_features.shape[0]

      #correct predictions
      correct+=(prediction==batch_labels).sum().item()

  accuracy=correct/total

  #return the accuracy to optimize
  return accuracy

In [ ]:
import optuna as opt

pruner=opt.pruners.MedianPruner(n_warmup_steps=5)

study=opt.create_study(direction='maximize',pruner=pruner)

In [ ]:
study.optimize(objective_function,n_trials=3)